In [9]:
import csv
import MySQLdb
import sys
import os
import subprocess
import genCSV as gc

In [ ]:
# Class kernel
class kernel:

    def __init__(self, entry):
        self.entry = entry

    def get_cid(self):
        return str(self.entry[0])

    def get_entry(self):
        return self.entry

    def kernel2csv(self):
        try:
            final = list([self.entry[0]] + [str(self.entry[1])] + [str(self.entry[2])] + [self.entry[7]])
            compressed = self.entry[8].split(" , ")
            parse_array = [i.split(" : ") for i in compressed]
            cprss = [fin[1] for fin in parse_array]
            final += cprss

            return final
        except Exception as e:
            #print("Entry len:", len(self.entry))
            print("\n" + str(e))
            exit(-1)

In [4]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [5]:
# Create a new kernel instance from the physical kernel
def compute_kernel(cid):
    print("\nComputing range of cid...", flush=True)
    kernels_array = []
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()
        
        for c in cid:
            query = "SELECT * FROM Compilations WHERE cid = %d and incremental_mod = 0" % c
            cursor.execute(query)
            entry = cursor.fetchone()
            
            if entry is not None:
                kernels_array.append(kernel(entry))
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(c), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()
    
    return kernels_array

In [6]:
def create_header():

    conn = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
    cursor = conn.cursor()

    get_prop = "SELECT name, type FROM Properties"
    # Extract properties
    cursor.execute(get_prop)
    types_results = list(cursor.fetchall())

    if len(types_results) == 0:
        print("\nError : Properties not present in database - You need to run Kanalyser first (https://github.com/TuxML/Kanalyser)")

    cursor.close()
    conn.close()

    # .config column names
    names = [""]*len(types_results)
    index = 0
    for (name, typ) in types_results:
        if name not in names:
            names[index] = name
            index += 1

    with open("config_bdd.csv", "w") as f:
        head = ("cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,LZMA-bzImage,LZMA-vmlinux,LZMA,XZ-bzImage,XZ-vmlinux,XZ,LZO-bzImage,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4," + ",".join(names)).split(",")
        writer = csv.DictWriter(f, head)
        writer.writeheader()

In [7]:
def to_csv(From, To):    
    values = gc.genCSV(0, From, To, incremental=False)
    
    # Correction for range(From, To) in order to have at least one value
    if From == To:
        To += 1

    cid_array = [c for c in range(From, To)]
    kernels_array = compute_kernel(cid_array)
    
    with open("config_bdd.csv", "a") as csv_file:
        writer = csv.writer(csv_file)
        
        if len(kernels_array) == 1:
            entry_val = kernels_array[0].kernel2csv()
            entry = entry_val + values[0]
            writer.writerow(entry)
        else:

            for num, k in enumerate(kernels_array):
                try:
                    entry_val = k.kernel2csv()
                    entry = entry_val + values[num]
                    #print("value:", values[num][7568])
                    writer.writerow(entry)
                except Exception as e:
                    print("Error at num:", num, flush=True)
                    print(str(e), flush=True)
                    exit(-1)

In [8]:
import bz2

def print_logs_configurations(cid):
    print("Printing logs of cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT stdlog_file, errlog_file, output_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        stdlog_file, errlog_file, output_file = cursor.fetchone()
            
        if (stdlog_file is None) or (errlog_file is None) or (output_file is None):
            print ("Unable to retrieve cid...")
            return
        
        print("STD LOG FILE")
        print(bz2.decompress(stdlog_file).decode('ascii'))
        print("#########")
        print("ERR LOG FILE")
        print("#########")
        print(bz2.decompress(errlog_file).decode()) # not ascii? 
        print("#########")
        print("OUTPUT LOG FILE")
        print("#########")
        print(bz2.decompress(output_file).decode('ascii'))
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(c), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

print_logs_configurations(116732) #78749: WANXL_BUILD_FIRMWARE # 79052 AIC7XXX_BUILD_FIRMWARE

Printing logs of cid= 116732


UnboundLocalError: local variable 'cursor' referenced before assignment

In [ ]:
def get_configuration_file(cid):
    print("Getting configuration file of cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT config_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        config_file = cursor.fetchone()
        
            
        if (config_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            return bz2.decompress(config_file[0]).decode('ascii')
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

# also check the absence of 'n' values (ie we check whether options have no "y" values) 
def check_configuration_file_specialized(cf):
    #if cf.find('CONFIG_SLOB=y') == -1:
    #    print("SLOB is no!")    
    return all(x in cf for x in ['CONFIG_64BIT=y', "CONFIG_OPTIMIZE_INLINING=y", "CONFIG_CC_OPTIMIZE_FOR_SIZE=y"]) and not any(x in cf for x in ["CONFIG_DEBUG_INFO_DWARF4=y", "CONFIG_AIC7XXX_BUILD_FIRMWARE=y", "CONFIG_AIC79XX_BUILD_FIRMWARE=y", "CONFIG_WANXL_BUILD_FIRMWARE=y", "CONFIG_LOCK_STAT=y", "CONFIG_X86_VSMP=y", "CONFIG_DEBUG_INFO=y", "CONFIG_DEBUG_INFO_SPLIT=y", "CONFIG_UBSAN_SANITIZE_ALL=y", "CONFIG_GCOV_PROFILE_ALL=y", "CONFIG_DEBUG_INFO_REDUCED=y", "CONFIG_RANDOMIZE_BASE=y", "CONFIG_X86_NEED_RELOCS=y", "CONFIG_KASAN_OUTLINE=y", "CONFIG_UBSAN_ALIGNMENT=y", "CONFIG_USB_SERIAL_OPTICON=y", "CONFIG_KASAN=y", "CONFIG_KCOV_INSTRUMENT_ALL=y", "CONFIG_XFS_DEBUG=y", "CONFIG_MAXSMP=y"])                                                                                                                                                    
# TODO: issues with these options:
# "CONFIG_SLOB=y" (seems not necessarily active)
## and these one that are apparently sometimes yes (despite being forced to no)
# "CONFIG_FW_LOADER_USER_HELPER"
# "CONFIG_STRICT_MODULE_RWX" 
## see below

# TODO: integrate in check_configuration_file_specialized
def check_spe_capricious_option(cf):
        return all(x in cf for x in ['CONFIG_SLOB=y']) and not any(x in cf for x in ["CONFIG_FW_LOADER_USER_HELPER=y", "CONFIG_STRICT_MODULE_RWX=y"])

cf1 = get_configuration_file(116733) #78749: WANXL_BUILD_FIRMWARE # 79052 AIC7XXX_BUILD_FIRMWARE
assert (check_configuration_file_specialized(cf1))


assert(check_configuration_file_specialized(get_configuration_file(116732)))
assert(check_configuration_file_specialized(get_configuration_file(116734)))
assert(check_configuration_file_specialized(get_configuration_file(116735)))
assert(check_configuration_file_specialized(get_configuration_file(116736)))

fail = 0
for i in range (123950, 124000):
    if (not check_configuration_file_specialized(get_configuration_file(i))):
        fail = fail + 1
        #print ("Specialization fails: ", fail)
print("failed Spe: ",fail/1000)
        


#get_configuration_file(119997)


In [25]:
def get_configuration_file(cid):
    print("Getting configuration file of cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT config_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        config_file = cursor.fetchone()
        
            
        if (config_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            return bz2.decompress(config_file[0]).decode('ascii')
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

# also check the absence of 'n' values (ie we check whether options have no "y" values) 
def check_configuration_file_specialized(cf):
    #if cf.find('CONFIG_SLOB=y') == -1:
    #    print("SLOB is no!")    
    return all(x in cf for x in ['CONFIG_64BIT=y', "CONFIG_OPTIMIZE_INLINING=y", "CONFIG_CC_OPTIMIZE_FOR_SIZE=y"]) and not any(x in cf for x in ["CONFIG_DEBUG_INFO_DWARF4=y", "CONFIG_AIC7XXX_BUILD_FIRMWARE=y", "CONFIG_AIC79XX_BUILD_FIRMWARE=y", "CONFIG_WANXL_BUILD_FIRMWARE=y", "CONFIG_LOCK_STAT=y", "CONFIG_X86_VSMP=y", "CONFIG_DEBUG_INFO=y", "CONFIG_DEBUG_INFO_SPLIT=y", "CONFIG_UBSAN_SANITIZE_ALL=y", "CONFIG_GCOV_PROFILE_ALL=y", "CONFIG_DEBUG_INFO_REDUCED=y", "CONFIG_RANDOMIZE_BASE=y", "CONFIG_X86_NEED_RELOCS=y", "CONFIG_KASAN_OUTLINE=y", "CONFIG_UBSAN_ALIGNMENT=y", "CONFIG_USB_SERIAL_OPTICON=y", "CONFIG_KASAN=y", "CONFIG_KCOV_INSTRUMENT_ALL=y", "CONFIG_XFS_DEBUG=y", "CONFIG_MAXSMP=y"])                                                                                                                                                    
# TODO: issues with these options:
# "CONFIG_SLOB=y" (seems not necessarily active)
## and these one that are apparently sometimes yes (despite being forced to no)
# "CONFIG_FW_LOADER_USER_HELPER"
# "CONFIG_STRICT_MODULE_RWX" 
## see below

# TODO: integrate in check_configuration_file_specialized
def check_spe_capricious_option(cf):
        return all(x in cf for x in ['CONFIG_SLOB=y']) and not any(x in cf for x in ["CONFIG_FW_LOADER_USER_HELPER=y", "CONFIG_STRICT_MODULE_RWX=y"])
    
def check_config_slob_dep(cf):
            if cf.find("CONFIG_EXPERT=y") != -1:
                print("config_expert  set to y checking others")
                if cf.find("CONFIG_SLOB=y") != -1:
                    print("config_slob set to y")
                if cf.find("CONFIG_FW_LOADER_USER_HELPER=y") != -1:
                    print("CONFIG_FW_LOADER_USER_HELPER set to y")
                if cf.find("CONFIG_FW_LOADER=y;") != 1:
                    print("CONFIG_FW_LOADER set to y")
                print("")
            elif cf.find("CONFIG_EXPERT=y") == -1:
                print("config_expert set to 'not set'")
                if cf.find("CONFIG_SLOB=y") == -1:
                    print("config_slob set to 'not set'")
                if cf.find("CONFIG_FW_LOADER_USER_HELPER=y") == -1:
                    print("CONFIG_FW_LOADER_USER_HELPER set to 'not set'")
                if cf.find("CONFIG_FW_LOADER=y") == -1:
                    print("CONFIG_FW_LOADER set to 'not set'")
                print("")
            else:
                print("config_slob and config_expert different value so not related")
#cf1 = get_configuration_file(116733) #78749: WANXL_BUILD_FIRMWARE # 79052 AIC7XXX_BUILD_FIRMWARE
#assert (check_configuration_file_specialized(cf1))
check_config_slob_dep(get_configuration_file(116732))
check_config_slob_dep(get_configuration_file(116734))
check_config_slob_dep(get_configuration_file(116735))
check_config_slob_dep(get_configuration_file(116736))

for i in range(123950, 124000):
    check_config_slob_dep(get_configuration_file(i))

#assert(check_configuration_file_specialized(get_configuration_file(116732)))
#assert(check_configuration_file_specialized(get_configuration_file(116734)))
#assert(check_configuration_file_specialized(get_configuration_file(116735)))
#assert(check_configuration_file_specialized(get_configuration_file(116736)))

#fail = 0
#for i in range (123950, 124000):
#    if (not check_configuration_file_specialized(get_configuration_file(i))):
#        fail = fail + 1
        #print ("Specialization fails: ", fail)
#print("failed Spe: ",fail/1000)
        


#get_configuration_file(119997)


Getting configuration file of cid= 116732
config_expert  set to y checking others
config_slob set to y
CONFIG_FW_LOADER_USER_HELPER set to y
CONFIG_FW_LOADER set to y

Getting configuration file of cid= 116734
config_expert  set to y checking others
config_slob set to y
CONFIG_FW_LOADER_USER_HELPER set to y
CONFIG_FW_LOADER set to y

Getting configuration file of cid= 116735
config_expert  set to y checking others
config_slob set to y
CONFIG_FW_LOADER_USER_HELPER set to y
CONFIG_FW_LOADER set to y

Getting configuration file of cid= 116736
config_expert  set to y checking others
config_slob set to y
CONFIG_FW_LOADER_USER_HELPER set to y
CONFIG_FW_LOADER set to y

Getting configuration file of cid= 123950
config_expert set to 'not set'
config_slob set to 'not set'

Getting configuration file of cid= 123951
config_expert  set to y checking others
config_slob set to y
CONFIG_FW_LOADER_USER_HELPER set to y
CONFIG_FW_LOADER set to y

Getting configuration file of cid= 123952
config_expert s

In [ ]:
# interestingly, for the "preset" options, options tinyconfig = options allnoconfig + our options spe
cf_allno_preset = get_configuration_file(124976)
check_configuration_file_specialized(cf_allno_preset), check_spe_capricious_option(cf_allno_preset)

cf_tinyconfig_x64 = get_configuration_file(88035)
check_configuration_file_specialized(cf_tinyconfig_x64), check_spe_capricious_option(cf_tinyconfig_x64)

# a "basic" make allnoconfig with only one presetting: X64
cf_allno_x64 = get_configuration_file(124978)
check_configuration_file_specialized(cf_allno_x64), check_spe_capricious_option(cf_allno_x64)

# TODO: compare .config (with the utility of Linux kernel)


In [ ]:
# UNCOMMENT if you want to extract/build a CSV out of the database
#if __name__ == "__main__":
    
    # Change values to append a new range of data from db to the csv
#    From = 20000 #100000 #30000 #15000 # 40510 56580 # 
#    To = 30000 #104500 #99400 #94390 #88036 #80054 # 78900 # 56593 # 
    
#    subprocess.run("rm config_bdd.csv", shell=True)
    
#    if not os.path.exists("config_bdd.csv"):
#        create_header()
#        print("config_bdd.csv created", flush=True)
#    
#    print("From %d to %d" % (From, To))
#    to_csv(From, To)
#    print("\nWritten!")
    
#    """
#    with open("config_bdd.csv", "r") as file:
#        t = file.readlines()
#        lines = []
#        for i in range(len(t)):
#            lines.append(t[i].split(","))
#    """